# ResNet50

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt 
import SimpleITK as sitk
import numpy as np
import tensorflow as tf
import sklearn
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from medpy.io import load
from skimage.measure import block_reduce
import scipy
import os
import seaborn as sns
import matplotlib.pyplot as plt 
import nilearn as nb
from nilearn import masking
import nibabel as nb


In [ ]:
preprocess_input_resnet = tf.keras.applications.resnet.preprocess_input
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

In [ ]:
t1 = sitk.ReadImage(os.getcwd() + "/doi_10_5061_dryad_5jv56__v20160424/Kim.2015.PLOS1.OCD-mm.data/mCCA+jICA_1K_results/HO1K_joint_comp_ica_feature_1_001.hdr")

In [ ]:
t1_array = sitk.GetArrayFromImage(t1)

In [ ]:
t1_array.shape

In [ ]:
plt.imshow(t1_array[44])

In [ ]:
91/2

In [ ]:
def normalize(arr):
    arr = arr + abs(np.amin(arr))
    assert np.amax(arr) != 0
    arr = arr / np.amax(arr)
    return arr

In [ ]:
X = []
name = []

for i in range(500):
    try :
        t1 = sitk.ReadImage(os.getcwd() + "/Data/FA" + str(1000+i) + ".nii")
        t2 = sitk.GetArrayFromImage(t1)
        t3 = []
        for j in range(0,90):
            result = scipy.ndimage.zoom(t2[j], 224/288)
            t3.append(result)
        X.append(t3)
        
    except:
        continue
    
for i in range(500):
    try :
        t1 = sitk.ReadImage(os.getcwd() + "/Data/FA" + str(2000+i) + ".nii")
        t2 = sitk.GetArrayFromImage(t1)
        t3 = []
        for j in range(0,90):
            result = scipy.ndimage.zoom(t2[j], 224/288)
            t3.append(result)
        X.append(t3)
        
    except:
        continue

print(len(X))


In [ ]:
len(t2)

In [ ]:
X[0][0]

2 3

In [ ]:
X[0] = np.array(X[0])

In [ ]:
X = np.array(X)
# 2
# (72, 40, 224, 224)

In [ ]:
def normalize(arr):
    arr = arr + abs(np.amin(arr))
    assert np.amax(arr) != 0
    arr = arr / np.amax(arr)
    return arr

# Display Images

In [ ]:
y = np.concatenate(( [0]*34, [1]*30 ))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=34)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
X_train = np.resize(X_train, ((40*44), 85, 71))
X_test = np.resize(X_test, ((40*20), 85, 71))

temp_y_train = np.array([])
for i in range(len(y_train)):
    for j in range(40):
        temp_y_train = np.append(temp_y_train, y_train[i])

temp_y_test = np.array([])
for i in range(len(y_test)):
    for j in range(40):
        temp_y_test = np.append(temp_y_test, y_test[i])

y_train = temp_y_train
y_test = temp_y_test

print(np.shape(X_train), np.shape(y_train), np.shape(X_test), np.shape(y_test))

In [ ]:
X_train_RGB = np.stack((X_train,)*3, axis=-1)

X_test_RGB = np.stack((X_test,)*3, axis=-1)

train_ds = preprocess_input_resnet(X_train_RGB)
test_ds = preprocess_input_resnet(X_test_RGB)

In [ ]:
base_model_resnet = tf.keras.applications.ResNet50(input_shape=(85, 71, 3),
                                               include_top=False,
                                               weights='imagenet')
base_model_resnet.trainable = False

In [ ]:
inputs = tf.keras.layers.Input((85, 71, 3))
x = base_model_resnet(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Flatten()(x)
predictions = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model_resnet = tf.keras.Model(inputs=inputs, outputs=predictions)

In [ ]:
base_learning_rate = 0.0001
model_resnet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

model_resnet.summary()

In [ ]:
history = model_resnet.fit(train_ds, y_train,
                    epochs=10,
                    validation_data=(test_ds, y_test))

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']
sns.set(font_scale=1)

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
base_model_resnet.trainable = True

# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model_resnet.layers))

# Fine-tune from this layer onwards
fine_tune_at = 10

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model_resnet.layers[:fine_tune_at]:
  layer.trainable = False

In [ ]:
model_resnet.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer = tf.keras.optimizers.Adam(learning_rate=base_learning_rate/10),
              metrics=['accuracy'])

In [ ]:
fine_tune_epochs = 10
total_epochs =  10 + fine_tune_epochs

history_fine = model_resnet.fit(train_ds, y_train,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=(test_ds, y_test), callbacks=[
                         tf.keras.callbacks.EarlyStopping(
                         monitor='val_loss',
                         patience=12,
                         mode='auto',
                         restore_best_weights=True,
)])

In [ ]:
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

initial_epochs = 10

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.ylim([0.5, 1])
plt.plot([initial_epochs,initial_epochs],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 1.0])
plt.plot([initial_epochs,initial_epochs],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
y_pred = []
y_true = y_test
# for i in X_test: print(i[:3000], "\n")

print(y_true)

for i in test_ds:
    i = np.expand_dims(i, axis=0)
    print(model_resnet.predict(i))
    j = np.round(model_resnet.predict(i))
    # print(j)
    y_pred.append((np.round(model_resnet.predict(i))))

y_pred

In [ ]:
#correct_labels = np.array(tf.concat([item for item in y_true], axis = 0))
predicted_labels = np.array(tf.concat([item for item in y_pred], axis = 0))

confusion_mtx = tf.math.confusion_matrix(y_true, predicted_labels)
plt.figure(figsize=(10, 8))
sns.set(font_scale=2)
sns.heatmap(confusion_mtx,
            xticklabels=["Healthy", "Depression"],
            yticklabels=["Healthy", "Depression"],
            annot=True, fmt='g', annot_kws={"size":40})
plt.xlabel('Prediction')
plt.ylabel('Truth')
plt.show()

print(confusion_mtx)

In [ ]:
model_resnet.save('RESNET_ocd_resting_state_dataset_t1_2d.h5')